In [ ]:
import os
import sys
from IPython import get_ipython
if get_ipython() is not None:
    base_dir = get_ipython().run_line_magic('pwd', '')
else:
    base_dir = os.getcwd()
sys.path.insert(0, str(base_dir))
import load
import torch as tch
import torch.nn as nn
import torch.optim as optim

In [3]:

def worker1(dict, list):
    l = len(list)
    if l in dict:
        dict[l]+=1
    else:
        dict[l] = 1
def collect(data, worker):
    result = dict()
    for i in data:
        worker(result, i)
    return result

In [ ]:
#arch
lr = 2e-3

class LSTM(nn.Module):
    def __init__(self, *args, **kwargs):
        self.emb_dim_in = 64
        self.emb_dim_out = 16
        self.in_vocab_embeddings = tch.randn((load.in_vocab_size, self.emb_dim_in))
        self.out_vocab_embeddings = tch.randn((load.out_vocab_size, self.emb_dim_out))
        super().__init__(*args, **kwargs)
    
    def vectorize(self, data_chunk):
        max_len = max([len(line) for line in data_chunk])
        in_ = tch.zeros((len(data_chunk), max_len, self.emb_dim_in))
        for line_idx, line in enumerate(data_chunk):
            for word_idx, word in enumerate(line):
                in_[line_idx, word_idx] = self.in_vocab_embeddings[word]
        return in_

def init_model(arch, LR):
    mod = arch()
    return mod, optim.Adam(mod.parameters(), LR)

model, optimizer = init_model(LSTM, lr)

In [ ]:

TRAIN_BATCHES = 1000# ~10 epochs
BATCH_SIZE = 200
for b in range(TRAIN_BATCHES):
    batch_data_in = load.encoded_in[b*BATCH_SIZE:(b+1)*BATCH_SIZE]
    batch_data_out = load.encoded_out[b*BATCH_SIZE:(b+1)*BATCH_SIZE]
    batch_data_in_vectorized = model.vectorize(batch_data_in)
    with tch.no_grad():
        model.zero_grad()
    #forward
    optimizer.step()
    
    